<a href="https://colab.research.google.com/github/aleanaliaaustral-web/labo2025v/blob/main/src/ensembles/594_TareaHogar_05_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea para el Hogar 05

Esta Tarea para el Hogar 05 se entrega el final de la cuarta clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 05 que será el miercoles 03 de septiembre

##  1. Overfitting the Public Leaderboard

Leer  https://medium.com/hmif-itb/overfitting-the-leaderboard-da25172ac62e
( 8 minutos )

## 2. Hiperparámetros del LightGBM

Los objetivos de esta tarea son:


*   Aumentar la rentabilidad de la campaña de marketing de retención proactiva de clientes.
*   Generar un mejor modelo optimizando sus hiperparámetros
*   Conceptual : investigar los mas relevantes hiperparámetros de LightGBM
*   Familiarizarse con la Bayesian Optimization, sus largos tiempos de corrida y opciones para reducirlos
*   Familiarizarse con el uso de máquinas virtuales de Google Colab
*   Ver un pipeline completo de optimización de hiperparámetros y puesta en producción

LightGBM cuenta con mas de 60 hiperparámetros, siendo posible utilizar 40 al mismo tiempo, aunque no razonable.
<br> La documentación oficial de los hiperparámetros de LightGBM es  https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters


Se lo alerta sobre que una Optimizacion Bayesiana lleva varias horas de corrida, y usted deberá correr VARIAS optimizaciones para descubrir cuales parámetros conviene optimizar.
<br> A pesar que la próxima clase es recien en viernes 01 de agosto, inicie la tarea con tiempo, aprenda a planificar estratégicamente sus corridas como un@ científ@  de datos.

Es necesario investigar cuales son los hiperparámetros de LightGBM que vale la pena optimizar en una Bayesian Optimization, ya que los realmente utiles son apenas un reducido subconjunto.
<br>Usted deberá investigar cuales son los hiperparámetros mas relevantes de LightGBM, su primer alternativa es preguntándole a su amigo con capacidades especiales ChatGPT o sus endogámicos familiares Claude, DeepSeek, Gemini, Grok, etc
<br> La segunda alternativa es la propia documentación de LightGBM  https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html


Adicionalmente podra buscar información como la que proveen esta diminuta muestra aleatoria de artículos ligeros:
* https://machinelearningmastery.com/light-gradient-boosted-machine-lightgbm-ensemble/
*  https://medium.com/@sarahzouinina/a-deep-dive-into-lightgbm-how-to-choose-and-tune-parameters-7c584945842e
*  https://www.kaggle.com/code/somang1418/tuning-hyperparameters-under-10-minutes-lgbm
*  https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702/


<br>  La muestra anterior se brinda a modo de ejemplo, usted deberá buscar muuuuchas  fuentes adicionales de información
<br> Tenga presente que LightGBM es el estado del arte en modelado predictivo para datasets estructurado, que son el 90% del trabajo del 95% de los Data Scientists en Argentina.

El desafío de esta tarea es:
* Qué hiperparparámetros conviene optimizar?  Las recomendaciones de los artículos ligeros es siempre sensata?  Sus autores realmente hicieron experimentos o son siemplemente escritores de entretenimiento carente de base científica?
* Elegidos los hiperparámetros, cual es el  <desde, hasta> que se debe utilizar en la Bayesian Optimization ?
* Realmente vale la pena optimizar 10 o 16 hiperparámetros al mismo tiempo ?  No resulta contraproducente una búsqueda en un espacio de tal alta dimensionalidad ?

#### 2.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 2.2 Optimizacion Hiperparámetros

Esta parte se debe correr con el runtime en lenguaje R Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

### 2.2.1 Inicio

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Nov 03 11:28:35 AM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660381,35.3,1454471,77.7,1454471,77.7
Vcells,1226627,9.4,8388608,64.0,1975128,15.1


### 2.2.2 Carga de Librerias

In [3]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: data.table

Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: yaml

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading requ

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [4]:
PARAM <- list()
PARAM$experimento <- 59405
PARAM$semilla_primigenia <- 900977


In [5]:
PARAM$kaggle$competencia <- "labo-i-2025-virtual-analista-sr"
PARAM$kaggle$cortes <- seq(10000, 12000, by= 500)

In [6]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.5

In [7]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)


 HIPERPARÁMETROS OPTIMIZADOS SEGÚN EL PAPER
### ============================================================================
# Basado en Imani & Arabnia (2023):
# - LightGBM alcanzó ROC AUC = 91% con optimización
# - Parámetros clave identificados:
#   * num_leaves = 1169
#   * max_depth = 10
#   * lambda_l1 = 2.689e-7 (regularización L1 muy pequeña)
#   * lambda_l2 = 7.238e-8 (regularización L2 muy pequeña)
#   * feature_fraction = 0.8708
#   * bagging_fraction = 0.6281
#   * bagging_freq = 7

Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [8]:
cat("\n")
cat("=========================================================================\n")
cat("CONFIGURACIÓN DE HIPERPARÁMETROS BASADA EN IMANI & ARABNIA (2023)\n")
cat("=========================================================================\n")
cat("Paper: LightGBM alcanzó ROC AUC = 91%\n")
cat("Optimizando 12 hiperparámetros críticos identificados en el paper\n")
cat("=========================================================================\n\n")

# Aqui se cargan los bordes de los hiperparametros de la BO
PARAM$hyperparametertuning$hs <- makeParamSet(

  # === PARÁMETROS BÁSICOS (originales del profesor) ===
  # makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 2000L), # Duplicate definition
  makeNumericParam("learning_rate", lower= 0.01, upper= 0.3),
  makeIntegerParam("num_leaves", lower= 31L, upper= 2048L),  # Paper: 1169
  makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 8000L), # Correct definition

  # === PARÁMETROS DE MUESTREO (críticos según paper) ===
  makeNumericParam("feature_fraction", lower= 0.4, upper= 1.0),  # Paper: 0.8708
  makeNumericParam("bagging_fraction", lower= 0.4, upper= 1.0),  # Paper: 0.6281
  makeIntegerParam("bagging_freq", lower= 1L, upper= 10L),       # Paper: 7

  # === ESTRUCTURA DEL ÁRBOL ===
  makeIntegerParam("max_depth", lower= 3L, upper= 15L),          # Paper: 10

  # === REGULARIZACIÓN (MUY IMPORTANTE para evitar overfitting) ===
  # El paper encontró que valores MUY PEQUEÑOS pero NO CERO son óptimos
  makeNumericParam("lambda_l1", lower= 1e-8, upper= 5.0),  # ✅ Debe decir 5.0
  makeNumericParam("lambda_l2", lower= 1e-8, upper= 5.0),  # ✅ Debe decir 5.0

  # === OTROS PARÁMETROS IMPORTANTES ===
  makeIntegerParam("min_child_samples", lower= 5L, upper= 100L), # Paper: 8
  makeNumericParam("min_gain_to_split", lower= 0.0, upper= 1.0)
)

cat("Hiperparámetros a optimizar:\n")
cat("  1. num_iterations     [100, 2000]      - Número de árboles\n")
cat("  2. learning_rate      [0.01, 0.3]      - Tasa de aprendizaje\n")
cat("  3. num_leaves         [31, 2048]       - Hojas por árbol (paper: 1169)\n")
cat("  4. min_data_in_leaf   [1, 8000]        - Mínimo de datos por hoja\n")
cat("  5. feature_fraction   [0.4, 1.0]       - % features por árbol (paper: 0.87)\n")
cat("  6. bagging_fraction   [0.4, 1.0]       - % datos por árbol (paper: 0.63)\n")
cat("  7. bagging_freq       [1, 10]          - Frecuencia bagging (paper: 7)\n")
cat("  8. max_depth          [3, 15]          - Profundidad máxima (paper: 10)\n")
cat("  9. lambda_l1          [1e-8, 10]       - Regularización L1 (paper: 2.7e-7)\n")
cat(" 10. lambda_l2          [1e-8, 10]       - Regularización L2 (paper: 7.2e-8)\n")
cat(" 11. min_child_samples  [5, 100]         - Min samples en nodo hijo (paper: 8)\n")
cat(" 12. min_gain_to_split  [0, 1]           - Ganancia mínima para split\n")
cat("\n")

# A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
# RECOMENDACIÓN BASADA EN EL PAPER Y TIEMPO DISPONIBLE:
# - 30 iteraciones: Insuficiente para 12 hiperparámetros
# - 50 iteraciones: Balance óptimo tiempo/calidad ⭐
# - 100+ iteraciones: Ideal pero requiere mucho tiempo

PARAM$hyperparametertuning$iteraciones <- 50  # ⚠️ Ajustar según tiempo disponible

cat("Iteraciones de Optimización Bayesiana:", PARAM$hyperparametertuning$iteraciones, "\n")
cat("Tiempo estimado: ~", round(PARAM$hyperparametertuning$iteraciones * 2.5 / 60, 1), "horas\n")
cat("Con 50 iteraciones tendrás un buen balance entre calidad y tiempo.\n")
cat("\n")


CONFIGURACIÓN DE HIPERPARÁMETROS BASADA EN IMANI & ARABNIA (2023)
Paper: LightGBM alcanzó ROC AUC = 91%
Optimizando 12 hiperparámetros críticos identificados en el paper

Hiperparámetros a optimizar:
  1. num_iterations     [100, 2000]      - Número de árboles
  2. learning_rate      [0.01, 0.3]      - Tasa de aprendizaje
  3. num_leaves         [31, 2048]       - Hojas por árbol (paper: 1169)
  4. min_data_in_leaf   [1, 8000]        - Mínimo de datos por hoja
  5. feature_fraction   [0.4, 1.0]       - % features por árbol (paper: 0.87)
  6. bagging_fraction   [0.4, 1.0]       - % datos por árbol (paper: 0.63)
  7. bagging_freq       [1, 10]          - Frecuencia bagging (paper: 7)
  8. max_depth          [3, 15]          - Profundidad máxima (paper: 10)
  9. lambda_l1          [1e-8, 10]       - Regularización L1 (paper: 2.7e-7)
 10. lambda_l2          [1e-8, 10]       - Regularización L2 (paper: 7.2e-8)
 11. min_child_samples  [5, 100]         - Min samples en nodo hijo (paper: 8)
 

In [9]:
PARAM$hyperparametertuning$hs <- makeParamSet(

  # === PARÁMETROS BÁSICOS (originales del profesor) ===
  makeIntegerParam("num_iterations", lower= 100L, upper= 2000L),
  makeNumericParam("learning_rate", lower= 0.01, upper= 0.3),
  makeIntegerParam("num_leaves", lower= 31L, upper= 2048L),
  makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 2000L),

  # === PARÁMETROS DE MUESTREO (críticos según paper) ===
  makeNumericParam("feature_fraction", lower= 0.4, upper= 1.0),
  makeNumericParam("bagging_fraction", lower= 0.4, upper= 1.0),
  makeIntegerParam("bagging_freq", lower= 1L, upper= 10L),

  # === ESTRUCTURA DEL ÁRBOL ===
  makeIntegerParam("max_depth", lower= 3L, upper= 15L),

  # === REGULARIZACIÓN (MUY IMPORTANTE para evitar overfitting) ===
  makeNumericParam("lambda_l1", lower= 1e-8, upper= 5.0),
  makeNumericParam("lambda_l2", lower= 1e-8, upper= 5.0),

  # === OTROS PARÁMETROS IMPORTANTES ===
  makeIntegerParam("min_child_samples", lower= 5L, upper= 100L),
  makeNumericParam("min_gain_to_split", lower= 0.0, upper= 0.5)
)

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

### 2.2.4  Preprocesamiento

In [10]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [11]:
# lectura del dataset

dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [12]:
dataset_train <- dataset[foto_mes %in% c(202107)]

In [13]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [14]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [15]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [16]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

[1] 83375

[1] 154

2.2.5 Configuracion Bayesian Optimization

In [17]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [18]:
# Aqui comienza la configuracion de la Bayesian Optimization
# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hyperparametertuning$hs, # defined at the beginning of the program
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)

2.2.6 Corrida Bayesian Optimization

In [19]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Mon Nov 03 11:47:54 AM 2025 AUC 0.925464231960594

Mon Nov 03 11:50:00 AM 2025 AUC 0.924818616246852

Mon Nov 03 11:53:16 AM 2025 AUC 0.925768324391676

Mon Nov 03 11:56:11 AM 2025 AUC 0.925812121735842

Mon Nov 03 11:57:48 AM 2025 AUC 0.928854592717457

Mon Nov 03 12:00:14 PM 2025 AUC 0.924415082863867

Mon Nov 03 12:01:56 PM 2025 AUC 0.922796502738473

Mon Nov 03 12:03:13 PM 2025 AUC 0.92608885636804

Mon Nov 03 12:04:27 PM 2025 AUC 0.922013065331092

Mon Nov 03 12:08:18 PM 2025 AUC 0.926396191350945

Mon Nov 03 12:09:42 PM 2025 AUC 0.930282486508586

Mon Nov 03 12:12:56 PM 2025 AUC 0.926389901333466

Mon Nov 03 12:14:12 PM 2025 AUC 0.926534899795999

Mon Nov 03 12:17:08 PM 2025 AUC 0.92917506552379

Mon Nov 03 12:18:17 PM 2025 AUC 0.922176107599982

Mon Nov 03 12:19:19 PM 2025 AUC 0.927399894205524

Mon Nov 03 12:22:39 PM 2025 AUC 0.925372857130719

Mon Nov 03 12:23:24 PM 2025 AUC 0.926944428065186

Mon Nov 03 12:27:24 PM 2025 AUC 0.9

In [20]:
# Attempt to install packages from CRAN
install.packages(c("primes", "rlist", "DiceKriging", "mlrMBO"), dependencies = TRUE)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘TH.data’, ‘plotrix’, ‘patchwork’, ‘libcoin’, ‘matrixStats’, ‘multcomp’, ‘lazyeval’, ‘iterators’, ‘Formula’, ‘plotmo’, ‘ggstats’, ‘igraph’, ‘mvtnorm’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘zoo’, ‘sandwich’, ‘plyr’, ‘rex’, ‘stringdist’, ‘rgenoud’, ‘foreach’, ‘doParallel’, ‘numDeriv’, ‘cmaesr’, ‘earth’, ‘emoa’, ‘GGally’, ‘gridExtra’, ‘kernlab’, ‘kknn’, ‘mco’, ‘party’, ‘randomForest’, ‘reshape2’, ‘covr’




In [21]:
install.packages("lightgbm")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



After running the installation cells above, run the cell below to load the required libraries. If any packages still fail to load, you may need to investigate their specific installation requirements.

In [22]:
# cargo las librerias que necesito
require("data.table")
require("parallel")
require("primes")
require("utils")
require("rlist")
require("yaml")
require("lightgbm")
require("DiceKriging")
require("mlrMBO")

In [23]:

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "num_iterations"    "learning_rate"     "num_leaves"       
 [4] "min_data_in_leaf"  "feature_fraction"  "bagging_fraction" 
 [7] "bagging_freq"      "max_depth"         "lambda_l1"        
[10] "lambda_l2"         "min_child_samples" "min_gain_to_split"
[13] "y"                 "dob"               "eol"              
[16] "error.message"     "exec.time"         "ei"               
[19] "error.model"       "train.time"        "prop.type"        
[22] "propose.time"      "se"                "mean"

In [24]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [25]:
write_yaml( PARAM, file="PARAM.yml")

In [26]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   num_iterations learning_rate num_leaves min_data_in_leaf feature_fraction
            <int>         <num>      <int>            <int>            <num>
1:           1743    0.01106477        515              391        0.4052658
   bagging_fraction bagging_freq max_depth lambda_l1 lambda_l2
              <num>        <int>     <int>     <num>     <num>
1:        0.7249838            9         7  1.663128 0.5086889
   min_child_samples min_gain_to_split
               <int>             <num>
1:                72         0.3104112
[1] 0.9322765


## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [27]:
setwd("/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [28]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [29]:
dataset_train <- dataset[foto_mes %in% c(202107)]

In [30]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

#### Final Training Hyperparameters

In [31]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 900977

$max_depth
[1] 7

$min_gain_to_split
[1] 0.3104112

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 1.663128

$lambda_l2
[1] 0.5086889

$max_bin
[1] 31

$bagging_fraction
[1] 0.7249838

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 1743

$learning_rate
[1] 0.01106477

$feature_fraction
[1] 0.4052658

$num_leaves
[1] 515

$min_data_in_leaf
[1] 391

$bagging_freq
[1] 9

$min_child_samples
[1] 72

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [35]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [36]:
  # entreno LightGBM

  modelo_final <- lgb.train(
    data= dtrain,
    param= param_normalizado
  )

In [37]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

  # entreno LightGBM

  modelo_final <- lgb.train(
    data= dtrain,
    param= param_normalizado
  )

In [38]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(modelo_final))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [39]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo_final, "modelo.txt" )

### Scoring

Aplico el modelo final a los datos del futuro

In [40]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

#### Tabla Prediccion

In [41]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

In [42]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
  Sys.sleep(45)
}

Warning message in dir.create("kaggle"):
“'kaggle' already exists”


Successfully submitted to LaboI 2025 virtual analista sr 
Successfully submitted to LaboI 2025 virtual analista sr 
Successfully submitted to LaboI 2025 virtual analista sr 
Successfully submitted to LaboI 2025 virtual analista sr 
Successfully submitted to LaboI 2025 virtual analista sr 


In [43]:
write_yaml( PARAM, file="PARAM.yml")

In [44]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Nov 03 06:34:35 PM 2025"

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar-05**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla

Seguramente usted realice varias corridas de este script con distintos conjuntos de hiperparámetros, siempre cambiandole el nombre al script  y también cambiando el nombre del experimento,  deberá TODAS esas corridas en distintas lineas de la  Google Sheet Colaborativa, hoja **TareaHogar-05**

Siéntase libre de agregar columnas a la hoja **TareaHogar-05**  en caso de ser necesario.